<a href="https://colab.research.google.com/github/Dicti93/MLSEM/blob/master/6.%20CNN/CNN_MLAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install torch-lr-finder
# from torch_lr_finder import LRFinder

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
import itertools
import gc
import copy
import re

from PIL import Image
from tabulate import tabulate
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_recall_curve, auc
from IPython.display import display, HTML

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, default_collate
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR


# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 230)  # Set the maximum width of the display

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
image_path = '/content/drive/My Drive/CNN/US_Pictures/'
print(os.listdir(image_path))

['490.2 App_M.bmp', '490.3 App_M.bmp', '490.4 App.bmp', '491.1 App_M.bmp', '491.2 App.bmp', '491.3 App_D.bmp', '492.1 App_M.bmp', '493.1 App_M.bmp', '493.2 App_M.bmp', '494.1 App.bmp', '494.2 App_M.bmp', '494.3 App.bmp', '494.4 App.bmp', '494.5 App.bmp', '494.6 App.bmp', '494.7 App_M.bmp', '494.8 App_D.bmp', '494.9 App_D.bmp', '496.1 App_M.bmp', '496.2 App_M.bmp', '497.1 App_M.bmp', '497.2 App_M.bmp', '498.1 App.bmp', '498.2 App_M.bmp', '499.1 App_M.bmp', '499.2 App.bmp', '499.3 App.bmp', '499.4 App.bmp', '499.5 App.bmp', '499.6 App_D.bmp', '499.7 App_M.bmp', '499.8 App_M.bmp', '499.9 App.bmp', '500.1 RLQ app not visible.bmp', '510.1 App_M.bmp', '502.1 App.bmp', '503.1 App.bmp', '503.2 app.bmp', '503.3 app.bmp', '503.4 App.bmp', '504.1 RLQ app not visible.bmp', '505.1 App_M.bmp', '505.2 App_M.bmp', '505.3 App_M.bmp', '505.4 App_M.bmp', '505.5 App_M.bmp', '505.6 App_M.bmp', '505.7 App_M.bmp', '506.1 App.bmp', '506.2 App.bmp', '506.3 App_M.bmp', '506.4 App.bmp', '506.5 App.bmp', '506.6 A

In [5]:
# !wget -O US_Pictures.zip https://zenodo.org/records/7669442/files/US_Pictures.zip
# !unzip -q US_Pictures.zip -d /content/US_Pictures/
# image_path = '/content/US_Pictures/US_Pictures/'
# print(os.listdir(image_path))

In [4]:
def display_images(image_path, num_images=5):
    # Ensure the function handles no image files gracefully
    files = [f for f in os.listdir(image_path) if f.endswith(('.png', '.bmp'))][:num_images]
    if not files:
        print("No image files found in the directory.")
        return

    fig, axs = plt.subplots(1, len(files), figsize=(15, 5))
    for i, file in enumerate(files):
        img = Image.open(os.path.join(image_path, file))
        axs[i].imshow(img)
        axs[i].axis('off')
        axs[i].set_title(file)
    plt.show()

display_images(image_path)

In [5]:
# Load the data
data_url = "https://raw.githubusercontent.com/Dicti93/MLSEM/master/2.%20Data%20and%20Description/data.csv"
tabular_data = pd.read_csv(data_url, delimiter= ';')

# Removing not needed target variables from data
data = tabular_data.drop(['Age', 'BMI', 'Sex', 'Height', 'Weight', 'Length_of_Stay', 'Management',
                          'Severity', 'Diagnosis_Presumptive', 'Alvarado_Score', 'Paedriatic_Appendicitis_Score',
                          'Appendix_on_US', 'Appendix_Diameter', 'Migratory_Pain', 'Lower_Right_Abd_Pain',
                          'Contralateral_Rebound_Tenderness', 'Coughing_Pain', 'Nausea', 'Loss_of_Appetite',
                          'Body_Temperature', 'WBC_Count', 'Neutrophil_Percentage', 'Segmented_Neutrophils',
                          'Neutrophilia', 'RBC_Count', 'Hemoglobin', 'RDW', 'Thrombocyte_Count', 'Ketones_in_Urine',
                          'RBC_in_Urine', 'WBC_in_Urine', 'CRP', 'Dysuria', 'Stool', 'Peritonitis',
                          'Psoas_Sign', 'Ipsilateral_Rebound_Tenderness', 'US_Performed', 'Free_Fluids',
                          'Appendix_Wall_Layers', 'Target_Sign', 'Appendicolith', 'Perfusion', 'Perforation',
                          'Surrounding_Tissue_Reaction', 'Appendicular_Abscess', 'Abscess_Location',
                          'Pathological_Lymph_Nodes', 'Lymph_Nodes_Location', 'Bowel_Wall_Thickening',
                          'Conglomerate_of_Bowel_Loops', 'Ileus', 'Coprostasis', 'Meteorism', 'Enteritis',
                          'Gynecological_Findings', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60','Unnamed: 61',
                          'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66','Unnamed: 67',
                          'Unnamed: 68'], axis=1)

# Mapping 'Diagnosis' to binary and handling missing values
data["Diagnosis"] = data["Diagnosis"].replace({'appendicitis': 1, 'no appendicitis': 0, })
data['Diagnosis'] = data['Diagnosis'].replace({np.nan: 0})

# Removing rows with NaN in 'US_Number' and converting columns to integer
data = data.dropna(subset=['US_Number'])
data['US_Number'] = data['US_Number'].astype(int)
data['Diagnosis'] = data['Diagnosis'].astype(int)

# Print the data to verify the conversion
print(data.head())

# Creating id_to_label and ensuring keys are strings
id_to_label = {int(row['US_Number']): row['Diagnosis'] for index, row in data.iterrows() if pd.notna(row['US_Number'])}

print(id_to_label)

   Diagnosis  US_Number
0          1        882
1          0        883
2          0        884
3          0        886
4          1        887
{882: 1, 883: 0, 884: 0, 886: 0, 887: 1, 888: 0, 889: 0, 890: 0, 891: 0, 893: 1, 894: 1, 895: 0, 896: 0, 897: 0, 900: 1, 903: 0, 904: 1, 905: 1, 909: 1, 910: 0, 911: 1, 913: 0, 914: 0, 916: 0, 917: 0, 918: 1, 921: 1, 922: 0, 924: 1, 925: 1, 926: 0, 927: 1, 928: 1, 929: 0, 930: 1, 931: 0, 933: 1, 934: 0, 935: 1, 936: 1, 938: 1, 939: 1, 940: 0, 942: 0, 943: 0, 945: 0, 946: 0, 947: 1, 948: 0, 949: 0, 950: 1, 951: 0, 952: 0, 953: 1, 954: 0, 955: 1, 959: 0, 961: 0, 962: 0, 963: 0, 965: 1, 967: 0, 968: 0, 971: 0, 973: 1, 974: 1, 976: 0, 977: 0, 978: 0, 981: 1, 982: 0, 983: 0, 986: 0, 988: 0, 989: 1, 991: 0, 992: 1, 733: 1, 735: 1, 736: 0, 737: 0, 738: 1, 739: 0, 740: 1, 743: 0, 745: 0, 746: 1, 748: 1, 749: 1, 750: 1, 751: 1, 752: 1, 753: 0, 754: 0, 755: 0, 756: 1, 757: 0, 759: 1, 760: 0, 761: 1, 765: 0, 769: 0, 771: 1, 772: 1, 775: 1, 777: 1, 779: 0,

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Input channels = 3 (RGB), output channels = 16
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer
        self.fc1 = nn.Linear(64 * 32 * 32, 512)  # Adjust the input features to match image size
        self.fc2 = nn.Linear(512, 2)  # Output layer: adjust according to the number of classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except the batch
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class ModifiedCNN(nn.Module):
    def __init__(self):
        super(ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)  # Increased dropout
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(x)  # Apply dropout
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class RevisedCNN(nn.Module):
    def __init__(self):
        super(RevisedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)  # Higher dropout after convolutional layers before the fully connected layer
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self)._init_()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class CustomResNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomResNet, self)._init_()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 1, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(64*BasicBlock.expansion, num_classes)

    def _make_layer(self, channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_channels, channels, stride))
            self.in_channels = channels * BasicBlock.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.adaptive_pool(out)  # Adaptive pooling to (1,1)
        out = torch.flatten(out, 1)  # Flatten all dimensions except batch
        out = self.linear(out)
        return out

class RestoredCNN(nn.Module):
    def __init__(self):
        super(RestoredCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout2d(0.2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout2d(0.3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = RestoredCNN()
print(model)

RestoredCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.1, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout2d(p=0.2, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout2d(p=0.3, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=131072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [7]:
class MedicalImagesDataset(Dataset):
    def __init__(self, root_dir, labels_dict, transform=None, augment=False):
        """
        Args:
            root_dir (string): Directory with all the image files.
            labels_dict (dict): Dictionary mapping image file names to labels.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.labels_dict = labels_dict
        self.transform = transform
        self.augment = augment
        self.filename_mapping = {}
        self.unique_us_numbers = set()
        self.diagnosis_count_per_image = defaultdict(int)
        self.augmented_count_per_image = defaultdict(int)
        self.num_images_found = 0
        table_data = []

# This section handles image loading, transformation, and optional augmentation.
        # Create a mapping between full filenames and image data
        for filename in os.listdir(root_dir):
            self.num_images_found += 1
            try:
                base_name = filename.split(' ')[0]  # Extract the numeric prefix
                match = re.match(r"([0-9]+(?:\.[0-9]+)?)", base_name)
                if match is None:
                    print(f"Warning: Filename '{filename}' does not match expected format.")
                    continue  # Skip this file
                base_name = match.group(1)
                image_number = int(base_name.split('.')[0])  # Convert to float to handle decimals
                # Use the image number as a key to get the corresponding label from labels_dict
                label = labels_dict.get(image_number)

                if label is not None:  # Only proceed if a valid label exists
                    self.unique_us_numbers.add(image_number)
                    self.diagnosis_count_per_image[label] += 1  # Increment the count for the diagnosis
                    image = Image.open(os.path.join(root_dir, filename)).convert('RGB')

# Apply initial transformation and handle class-specific augmentation.
                    # Store the tensor along with the full filename
                    if self.transform:
                        processed_image = self.transform(image)
                    self.filename_mapping[base_name] = (processed_image, label)
                    self.unique_us_numbers.add(image_number)
                    table_data.append((image_number, base_name, label, type(image_number)))

                    if self.augment:
                      times = 2 if label == 0  else 0
                      augmented_images = self.augment_image(image, times, base_name)
                      for aug_filename, aug_image in augmented_images.items():
                          self.filename_mapping[aug_filename] = (aug_image, label)
                          self.augmented_count_per_image[label] += 1

# Exception handling for files with unexpected naming conventions enhances robustness.
            except ValueError:
                # This handles cases where the filename does not start with an integer
                print(f"Skipping file with non-standard naming: {filename}")

        # Sort the table data by the "Image Number" column
        sorted_table_data = sorted(table_data, key=lambda x: x[0])

        # Print the sorted table using tabulate after iterating over all images
        # print(tabulate(sorted_table_data, headers=['Image Number', 'Filename', 'Diagnosis', 'Type']))
        self.print_dataset_statistics()

# Compute and display dataset statistics to verify correct processing and balance.
    def print_dataset_statistics(self):
        print(f"Total images found before matching: {self.num_images_found}")
        print(f"Total us_numbers successfully mapped to images: {len(self.unique_us_numbers)}")
        print(f"Total images successfully mapped to a US_NUMBER: {len(self.filename_mapping)}")
        print(f"Original images: {sum(self.diagnosis_count_per_image.values())}")
        print(f"Augmented images: {sum(self.augmented_count_per_image.values())}")

        for label, count in self.diagnosis_count_per_image.items():
            aug_count = self.augmented_count_per_image[label]
            total_count = count + aug_count
            ratio = aug_count / float(count) if count != 0 else 0
            print(f"Label {label}: Original {count}, Augmented {aug_count}, Total {total_count}, Ratio {ratio:.2f}")

# Dynamic image augmentation based on a predefined set of possible transformations.
    def augment_image(self, image, times, original_filename):
    # All possible transformations
        possible_transformations = [
            lambda x: TF.rotate(x, -10),  # Rotate by various degrees
            lambda x: TF.rotate(x, -5),   # Rotate by various degrees
            lambda x: TF.rotate(x, 5),    # Rotate by various degrees
            lambda x: TF.rotate(x, 10),   # Rotate by various degrees
            lambda x: TF.affine(x, translate=(0.1, 0.1), scale=1.05, angle=0, shear=0),   # Enlargement
            lambda x: TF.affine(x, translate=(0.1, 0.1), scale=1.1, angle=0, shear=0),    # Enlargement
            lambda x: TF.affine(x, translate=(-0.1, -0.1), scale=0.95, angle=0, shear=0), # Shrinking
            lambda x: TF.affine(x, translate=(-0.1, -0.1), scale=0.9, angle=0, shear=0),  # Shrinking
            lambda x: TF.affine(x, angle=0, translate=(0, 0), scale=1.0, shear=5),        # Shearing
        ]

        # Ensure we do not pick more transformations than available
        num_transforms = min(len(possible_transformations), times)
        # Select unique transformations without replacement
        selected_transforms = random.sample(possible_transformations, num_transforms)
        augmented_images = {}
        for i, transform in enumerate(selected_transforms):
            if self.transform:
                aug_image = self.transform(image)  # Additional transformations like normalization
            augmented_images[f"{original_filename}_{i+1}"] = aug_image

        return augmented_images

# Standard dataset methods for PyTorch compatibility.
    def __len__(self):
        return len(self.filename_mapping)

    def __getitem__(self, idx):
        # Retrieve the data point using the filename mapping
        filename = list(self.filename_mapping.keys())[idx]
        image, label = self.filename_mapping[filename]
        return image, label, filename

In [ ]:
# Define image transformations to standardize and normalize images for consistent model input.
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizse
])
# Initialize the dataset with the specified directory, label mappings, transformations, and augmentation enabled.
dataset = MedicalImagesDataset(image_path, id_to_label, transform, augment=True)

In [ ]:
# Print first entries of the dataset to inspect their structure
for idx in range(1):
    print(dataset[idx])

In [ ]:
# Deep copy of datasets to preserve originals
filename_mapping_copy = copy.deepcopy(dataset.filename_mapping)
labels_dict_copy = copy.deepcopy(dataset.labels_dict)

# Initialize a dictionary to count images per US_NUMBER
image_counts = defaultdict(int)

# Collect initial image counts
for filename in filename_mapping_copy:
    try:
        us_number = int(filename.split('.')[0])
        image_counts[us_number] += 1
    except ValueError as e:
        print(f"Skipping filename with parsing issue: {filename}. Error: {e}")

# Dictionary for reassigning labels and renaming images, and set threshold
threshold = 10
reassigned_filenames = {}
reassigned_labels = {}

# Collect keys to iterate over to prevent runtime error due to dictionary size change
us_numbers_to_process = [us_number for us_number, count in image_counts.items() if count > threshold]

# Reassigning process with dynamic suffix management
for us_number in us_numbers_to_process:
    suffix_files = defaultdict(list)
    for filename in list(filename_mapping_copy):  # Use a list to safely remove items from dictionary
        if int(filename.split('.')[0]) == us_number and '_' in filename:
            parts = filename.split('_')[1]
            if parts[0].isdigit():
                suffix = int(parts[0])
                suffix_files[suffix].append(filename)

    # Assign new us_number for each suffix
    for suffix, files in suffix_files.items():
        new_us_number = (suffix + 1) * 1000 + us_number  # Create a new us_number based on suffix
        for filename in files:
            new_filename = filename.replace(str(us_number), str(new_us_number))
            reassigned_filenames[new_filename] = filename_mapping_copy.pop(filename)
            original_label = labels_dict_copy.get(us_number, "Unknown label")
            reassigned_labels[new_filename] = original_label
            labels_dict_copy[new_us_number] = original_label
            image_counts[new_us_number] += 1

# Update the datasets with new mappings
filename_mapping_copy.update(reassigned_filenames)

# Recalculate counts and prepare for output
new_image_counts = defaultdict(int)
for filename in filename_mapping_copy:
    try:
        us_number = int(filename.split('.')[0])
        new_image_counts[us_number] += 1
    except ValueError as e:
        print(f"Skipping filename with parsing issue: {filename}. Error: {e}")

# Prepare data for tabulation
table_data = [
    [us_number, image_counts[us_number], new_image_counts[us_number], labels_dict_copy.get(us_number)]
    for us_number in sorted(new_image_counts)
    ]

# Print the final table
# print(tabulate(table_data, headers=["US_NUMBER", "Old Count", "New Count", "Label"], tablefmt="grid"))

# Split table_data into three parts
split_size = len(table_data) // 4
part1 = table_data[:split_size]
part2 = table_data[split_size:2*split_size]
part3 = table_data[2*split_size:3*split_size]
part4 = table_data[3*split_size:]

# Helper function to convert data to HTML table with borders
def data_to_html_table(data):
    html = '<table style="width: 100%; border-collapse: collapse;">'
    html += '<tr style="border: 1px solid black;">' \
            '<th style="border: 1px solid black;">US_NUMBER</th>' \
            '<th style="border: 1px solid black;">Old Count</th>' \
            '<th style="border: 1px solid black;">New Count</th>' \
            '<th style="border: 1px solid black;">Label</th></tr>'
    for row in data:
        html += '<tr>' + ''.join(f'<td style="border: 1px solid black;">{cell}</td>' for cell in row) + '</tr>'
    html += '</table>'
    return html

# Display the three parts side by side using HTML and CSS
display(HTML(f'''
<div style="display: flex; justify-content: center; width: 100%;">
    <div style="flex: 1; padding: 5px;"><b>Part 1:</b><br>{data_to_html_table(part1)}</div>
    <div style="flex: 1; padding: 5px;"><b>Part 2:</b><br>{data_to_html_table(part2)}</div>
    <div style="flex: 1; padding: 5px;"><b>Part 3:</b><br>{data_to_html_table(part3)}</div>
    <div style="flex: 1; padding: 5px;"><b>Part 4:</b><br>{data_to_html_table(part4)}</div>
</div>
'''))

In [ ]:
# Print the count of pictures in the original and copied filename mappings
print("Count of pictures in original filename_mapping:", len(dataset.filename_mapping))
print("Count of pictures in filename_mapping_copy:", len(filename_mapping_copy))

# Function to count unique US numbers
def count_us_numbers(filename_mapping):
    us_numbers = set()
    for filename in filename_mapping:
        try:
            us_number = int(filename.split('.')[0])
            us_numbers.add(us_number)
        except ValueError:
            continue  # Skip filenames that do not start with an integer
    return len(us_numbers)

# Calculate unique US number counts after processing
final_original_us_numbers_count = count_us_numbers(dataset.filename_mapping)
final_copied_us_numbers_count = count_us_numbers(filename_mapping_copy)

# Print the final counts
print("Final count of unique US numbers in original filename_mapping:", final_original_us_numbers_count)
print("Final count of unique US numbers in filename_mapping_copy:", final_copied_us_numbers_count)

In [ ]:
# Custom collate function
def custom_collate(batch):
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    filenames = [item[2] for item in batch]

    # Find the maximum sequence length in the batch
    max_sequence_length = max(len(image) for image in images)

    # Pad images to the same size
    padded_images = []
    for image_sequence in images:
        num_padding = max_sequence_length - len(image_sequence)
        padded_sequence = torch.nn.functional.pad(image_sequence, (0, num_padding))
        padded_images.append(padded_sequence)

    # Stack padded images
    stacked_images = torch.stack(padded_images)

    return stacked_images, torch.tensor(labels), filenames

In [ ]:
us_numbers = []
labels = []

dataset.filename_mapping.update(filename_mapping_copy)
dataset.labels_dict.update(labels_dict_copy)

# Loop over each entry in the filename mapping
for filename, (image, label) in dataset.filename_mapping.items():
    image_number = int(filename.split('.')[0])
    us_numbers.append(image_number)
    labels.append(label)

# Convert lists to numpy arrays for proper indexing
us_numbers = np.array(us_numbers)
labels = np.array(labels)

# Create the splitter
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get indices for the splits
train_inds, val_inds = next(gss.split(X=labels, groups=us_numbers))

# Extract datasets for training and validation using list comprehension
train_dataset = [dataset[i] for i in train_inds]
val_dataset = [dataset[i] for i in val_inds]

# Create DataLoader for both training and validation sets
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

In [ ]:
# Collect filenames and labels from train_dataset
train_filenames = [filename for _, label, filename in train_dataset]
train_labels = [label for _, label, filename in train_dataset]

# Collect filenames and labels from val_dataset
val_filenames = [filename for _, label, filename in val_dataset]
val_labels = [label for _, label, filename in val_dataset]

# Create DataFrames
train_df = pd.DataFrame({'Filename': train_filenames, 'Label': train_labels})

val_df = pd.DataFrame({'Filename': val_filenames, 'Label': val_labels})

# Sort by Filename
train_df_sorted = train_df.sort_values(by='Filename', ascending=True)
val_df_sorted = val_df.sort_values(by='Filename', ascending=True)

# Splitting the DataFrame into n parts
split_size = len(train_df_sorted) // 7
tr1 = train_df_sorted.iloc[:split_size]
tr2 = train_df_sorted.iloc[split_size:2*split_size]
tr3 = train_df_sorted.iloc[2*split_size:3*split_size]
tr4 = train_df_sorted.iloc[3*split_size:4*split_size]
tr5 = train_df_sorted.iloc[4*split_size:5*split_size]
tr6 = train_df_sorted.iloc[5*split_size:6*split_size]
tr7 = train_df_sorted.iloc[6*split_size:]

split_size = len(val_df_sorted) // 2
val1 = val_df_sorted.iloc[:split_size]
val2 = val_df_sorted.iloc[split_size:]

# Display train_df_sorted and val_df_sorted side by side using HTML and CSS with reduced padding and centered alignment
display(HTML('<div style="display: flex; justify-content: center; width: 100%;">'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 1:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 2:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 3:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 4:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 5:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 6:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Train_df part 7:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Val_df part 1:</b><br>{}</div>'
             '<div style="flex: 1; padding: 5px;"><b>Val_df part 2:</b><br>{}</div>'
             '</div>'.format(tr1.to_html(index=False),
                             tr2.to_html(index=False),
                             tr3.to_html(index=False),
                             tr4.to_html(index=False),
                             tr5.to_html(index=False),
                             tr6.to_html(index=False),
                             tr7.to_html(index=False),
                             val1.to_html(index=False),
                             val2.to_html(index=False))))

In [ ]:
# Count occurrences of each diagnosis in the training set
train_diagnosis_counts = train_df_sorted['Label'].value_counts()

# Count occurrences of each diagnosis in the validation set
val_diagnosis_counts = val_df_sorted['Label'].value_counts()

# Print the counts
print("Diagnosis Distribution in Training Data:")
print(train_diagnosis_counts)
print("\nDiagnosis Distribution in Validation Data:")
print(val_diagnosis_counts)

In [ ]:
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 6))  # Adjust figsize as needed

# Plot for Training Data
sns.countplot(x='Label', data=train_df_sorted, ax=axs[0])
axs[0].set_title('Diagnosis Distribution in Training Data by Image')
axs[0].set_xlabel('Diagnosis')
axs[0].set_ylabel('Count')

# Plot for Validation Data
sns.countplot(x='Label', data=val_df_sorted, ax=axs[1])
axs[1].set_title('Diagnosis Distribution in Validation Data by Image')
axs[1].set_xlabel('Diagnosis')
axs[1].set_ylabel('Count')

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()

In [ ]:
# Function to extract US_NUMBER from Filename
def extract_us_number(filename):
    return int(filename.split('.')[0])

# Apply the function to train and validation DataFrames
train_df_sorted['US_NUMBER'] = train_df_sorted['Filename'].apply(extract_us_number)
val_df_sorted['US_NUMBER'] = val_df_sorted['Filename'].apply(extract_us_number)

# Aggregate by US_NUMBER and get the most frequent Label for cases of multiple images per US_NUMBER
train_diagnosis = train_df_sorted.groupby('US_NUMBER')['Label'].agg(lambda x: x.mode()[0])
val_diagnosis = val_df_sorted.groupby('US_NUMBER')['Label'].agg(lambda x: x.mode()[0])

# Reset index to turn series back into DataFrame for plotting
train_diagnosis = train_diagnosis.reset_index()
val_diagnosis = val_diagnosis.reset_index()

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 6))  # Adjust figsize as needed

# Plot for Training Data by US_NUMBER
sns.countplot(x='Label', data=train_diagnosis, ax=axs[0])
axs[0].set_title('Diagnosis Distribution by US_NUMBER in Training Data')
axs[0].set_xlabel('Diagnosis')
axs[0].set_ylabel('Count of US_NUMBERS')

# Plot for Validation Data by US_NUMBER
sns.countplot(x='Label', data=val_diagnosis, ax=axs[1])
axs[1].set_title('Diagnosis Distribution by US_NUMBER in Validation Data')
axs[1].set_xlabel('Diagnosis')
axs[1].set_ylabel('Count of US_NUMBERS')

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()

In [ ]:
# Define loss function (criterion) and optimiser
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()

# optimiser = optim.Adam(model.parameters(), lr=0.05)  # start with a very low LR
# scheduler = ReduceLROnPlateau(optimiser, mode='min', factor=0.5, patience=3)
optimiser = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # Configure the optimizer
scheduler = CyclicLR(optimiser, base_lr=0.001, max_lr=0.01, step_size_up=2000, mode='triangular')


# Training loop
num_epochs = 25
best_validation_loss = float('inf')
patience = 7  # Number of epochs to wait for improvement
patience_counter = 0  # Initialize patience counter once before the loop
increasing_loss_count = 0  # Counter for consecutive increases in validation loss
last_validation_loss = float('inf')  # Last epoch's validation loss for comparison

for epoch in range(num_epochs):
    model.train()
  # Calculate training loss
    running_loss = 0.0
    correct_train_predictions = 0
    total_train_examples = 0

    for images, labels, filenames in train_loader:
        optimiser.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
        # Step the scheduler with CyclicLR
        scheduler.step()

        # Calculate number of correct predictions
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train_predictions += (predicted == labels).sum().item()
        total_train_examples += labels.size(0)

    # Calculate training accuracy
    train_accuracy = correct_train_predictions / total_train_examples

    # Calculate validation loss
    validation_loss = 0.0
    with torch.no_grad():
        for val_images, val_labels, _ in val_loader:
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            validation_loss += val_loss.item()

    # Step the scheduler with ReduceLROnPlateau
    # scheduler.step(validation_loss)

    # Retrieve and print the last learning rate
    last_lr = scheduler.get_last_lr()
    print(f'Current learning rate: {last_lr}')

    # Print epoch statistics
    average_train_loss = running_loss / len(train_loader)
    average_validation_loss = validation_loss / len(val_loader)
    print(f'Increasing Loss Count: {increasing_loss_count}, Early Stopping Patiance Counter: {patience_counter}')
    print(f'Epoch {epoch + 1}, Train Loss: {average_train_loss}, Validation Loss: {average_validation_loss}, Train Accuracy: {train_accuracy:.2f}')


    # Check for improvement in validation loss and increment counters appropriately
    if average_validation_loss < best_validation_loss:
        best_validation_loss = average_validation_loss
        patience_counter = 0 # Reset since we have seen improvement
        increasing_loss_count = 0  # Reset since validation loss improved
        torch.save(model.state_dict(), 'best_model.pth')  # Save the model if validation loss improved
    else:
        patience_counter += 1

    # Check for rise in validation loss and increment counters appropriately
        if last_validation_loss < average_validation_loss:
            increasing_loss_count += 1
        else:
            increasing_loss_count = 0  # Reset increasing loss count on any improvement or no change

    # Update the last validation loss at the end of the loop
    last_validation_loss = average_validation_loss

    # Check if early stopping criteria met
    if increasing_loss_count >= patience:
        print(f'Early stopping triggered due to increased validation loss in {patience} consecutive epochs at epoch {epoch + 1}.')
        break

    if patience_counter >= patience:
        print(f'Early stopping after {patience} epochs without improvement.')
        break

print('Finished Training')

In [ ]:
model.eval()

# Lists to store predictions, ground truth labels, and filenames
all_probabilities = []
all_predictions = []
all_labels = []
all_filenames = []
all_outputs = []

# Iterate over the validation dataset
for images, labels, filenames in val_loader:
    with torch.no_grad():  # No need to compute gradients during inference
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get the predicted class indices
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, 1]  # Get probabilities for class 1

        # Store predictions, labels, filenames, and outputs
        all_probabilities.extend(probabilities.tolist())
        all_predictions.extend(predicted.tolist())
        all_labels.extend(labels.tolist())
        all_filenames.extend(filenames)
        all_outputs.extend(outputs.tolist())

# Convert predictions and labels to numpy arrays for further analysis if needed
all_probabilities = np.array(all_probabilities)
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calculate accuracy
accuracy = np.mean(all_predictions == all_labels)
print(f'Validation Accuracy: {accuracy}')

# Compute ROC AUC Score
roc_auc = roc_auc_score(all_labels, all_probabilities)
print(f'ROC AUC Score: {roc_auc}')

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(all_labels, all_probabilities)

# Define class labels
class_labels = ['No Appendicitis', 'Appendicitis']
# Calculate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

precision, recall, _ = precision_recall_curve(all_labels, all_probabilities)
pr_auc = auc(recall, precision)

# Set up a 1x3 subplot layout
fig, axs = plt.subplots(1, 3, figsize=(24, 6))  # Adjust figsize as needed to accommodate three plots

# Plot 1: Precision-Recall curve
axs[0].plot(recall, precision, label=f'Precision-Recall curve (area = {pr_auc:.2f})')
axs[0].set_xlabel('Recall')
axs[0].set_ylabel('Precision')
axs[0].set_title('Precision-Recall Curve')
axs[0].legend(loc="lower left")

# Plot 2: Confusion Matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels, ax=axs[1])
axs[1].set_xlabel('Predicted Label')
axs[1].set_ylabel('True Label')
axs[1].set_title('Confusion Matrix')

# Plot 3: ROC curve
axs[2].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
axs[2].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axs[2].set_xlabel('False Positive Rate')
axs[2].set_ylabel('True Positive Rate')
axs[2].set_title('Receiver Operating Characteristic')
axs[2].legend(loc="lower right")

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()

# Store each prediction along with filenames and outputs
predictions_data = []
for i in range(len(all_predictions)):
    predictions_data.append({'Filename': all_filenames[i],
                             'Ground Truth': all_labels[i],
                             'Prediction': all_predictions[i],
                             'Output': all_outputs[i]})
predictions_df = pd.DataFrame(predictions_data)
# print(predictions_df)